In [3]:
from requests.auth import HTTPProxyAuth
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import undetected_chromedriver as uc
from selenium.webdriver.common.proxy import Proxy, ProxyType
import time
import csv

proxy_host = '184.174.24.237'  
proxy_port = '6813'              
proxy_username = 'fjhdetvm'  
proxy_password = 'dj626ier4ofb'  
PROXY = "161.123.101.231:6857"

chrome_options = uc.ChromeOptions()
chrome_options.add_argument('--proxy-server=%s' % PROXY)
driver = uc.Chrome(options=chrome_options)

def extract_product_urls():
 
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CLASS_NAME, 'linkHover')))
    product_urls = []
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    for link in soup.find_all('a', class_='linkHover'):
        product_url = link.get('href')
        if product_url:
            product_urls.append(product_url)
    
    return product_urls

def product_data(product_url):
    driver.get(product_url)
    time.sleep(2) 
    soup_product = BeautifulSoup(driver.page_source, 'html.parser')
    product_info = {}
    try:
        
        all_div = soup_product.find_all('div', class_='carouselListItem')
        product_info['Image URL'] = all_div[0].find('img')['src'] if all_div else None

        product_title_cont = soup_product.find('div', class_='productTitleCont')
        if product_title_cont:
            brand = product_title_cont.find('a', class_='brandNameLink')
            product_info['Brand'] = brand.text.strip() if brand else None
            
            title_span = product_title_cont.find('span')
            product_info['Product Title'] = title_span.text.strip() if title_span else None
        else:
            print("Product title container not found.")
            

       
        price = soup_product.find('span', class_='currPrice')
        product_info['Price'] = price.text.strip() if price else None

        
        details_panel = soup_product.find('div', id='accordion__panel-1')
        if details_panel:
            location = details_panel.find('p').text
            product_info['Location'] = location.split(':')[-1].strip()
            item_details = details_panel.find_all('p')
            for detail in item_details:
                if 'Item #' in detail.text:
                    product_info['Item Number'] = detail.text.split(':')[-1].strip()
                elif 'Year:' in detail.text:
                    product_info['Year'] = detail.text.split(':')[-1].strip()

        condition_button = soup_product.find('button', class_='additionalText')
        if condition_button:
            product_info['Condition'] = condition_button.text.strip() if condition_button else None

        shipping_info_panel = soup_product.find('div', id='accordion__panel-0')
        if shipping_info_panel:
            shipping_info = shipping_info_panel.find_all('p', class_='shippingInfo')
            product_info['Shipping Info'] = [info.text.strip() for info in shipping_info]
            
            comes_with_heading = shipping_info_panel.find('p', class_='bodyHeading')
            if comes_with_heading:
                items_list = shipping_info_panel.find('ul', class_='bodyList')
                if items_list:
                    product_info['Comes With'] = [item.text.strip() for item in items_list.find_all('li')]
        size_panel = soup_product.find('div', id='accordion__panel-4')
        if size_panel:
            size_info = size_panel.find_all('p')
            product_info['Size'] = {size.text.split(':')[0]: size.text.split(':')[1].strip() for size in size_info}
 
        description_panel = soup_product.find('div', id='accordion__panel-2')
        if description_panel:
            description = description_panel.find('p')
            product_info['Description'] = description.text.strip() if description else None

        return product_info

    except Exception as e:
        print(f"Error fetching product details: {e}")
        pass
start_page_url = 'https://www.fashionphile.com/shop/brands/hermes'
csv_file_path = 'fashionphile_product_data.csv'
with open(csv_file_path, 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['Image URL', 'Brand', 'Product Title', 'Price', 'Location', 'Item Number', 'Year', 'Condition','Shipping Info','Comes With', 'Size', 'Description']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    all_product_info = []
    current_page = 1

    while True:
        page_url = f"{start_page_url}?page={current_page}"
        driver.get(page_url)
        time.sleep(2) 

        product_urls = extract_product_urls()
        print(f"Found {len(product_urls)} product URLs on page {current_page}.")
        
        for url in product_urls:
            product_info = product_data(url)
            if product_info:
                all_product_info.append(product_info)
                writer.writerow(product_info)
                csvfile.flush()  
                    
            length = len(all_product_info)
            print(f"Current CSV length : {length}")                
        if len(product_urls) == 0:
            print("No more products found.")
            break

        current_page += 1

        print(f"Data saved to {csv_file_path}")


Found 120 product URLs on page 1.
Current CSV length : 1
Current CSV length : 2
Current CSV length : 3
Current CSV length : 4
Current CSV length : 5
Current CSV length : 6
Current CSV length : 7
Current CSV length : 8
Current CSV length : 9
Current CSV length : 10
Current CSV length : 11
Current CSV length : 12
Current CSV length : 13
Current CSV length : 14
Current CSV length : 15
Current CSV length : 16
Current CSV length : 17
Product title container not found.
Current CSV length : 18
Current CSV length : 19
Current CSV length : 20
Current CSV length : 21
Current CSV length : 22
Current CSV length : 23
Current CSV length : 24
Current CSV length : 25
Current CSV length : 26
Current CSV length : 27
Current CSV length : 28
Current CSV length : 29
Current CSV length : 30
Current CSV length : 31
Current CSV length : 32
Current CSV length : 33
Current CSV length : 34
Current CSV length : 35
Current CSV length : 36
Current CSV length : 37
Current CSV length : 38
Current CSV length : 39
Curr

Current CSV length : 325
Current CSV length : 326
Error fetching product details: list index out of range
Current CSV length : 326
Current CSV length : 327
Current CSV length : 328
Current CSV length : 329
Current CSV length : 330
Current CSV length : 331
Current CSV length : 332
Current CSV length : 333
Current CSV length : 334
Current CSV length : 335
Current CSV length : 336
Current CSV length : 337
Current CSV length : 338
Current CSV length : 339
Current CSV length : 340
Current CSV length : 341
Current CSV length : 342
Current CSV length : 343
Current CSV length : 344
Current CSV length : 345
Current CSV length : 346
Current CSV length : 347
Current CSV length : 348
Current CSV length : 349
Current CSV length : 350
Current CSV length : 351
Current CSV length : 352
Current CSV length : 353
Current CSV length : 354
Current CSV length : 355
Current CSV length : 356
Current CSV length : 357
Current CSV length : 358
Current CSV length : 359
Data saved to fashionphile_product_data.csv


Current CSV length : 641
Current CSV length : 642
Current CSV length : 643
Current CSV length : 644
Current CSV length : 645
Current CSV length : 646
Current CSV length : 647
Current CSV length : 648
Current CSV length : 649
Current CSV length : 650
Current CSV length : 651
Current CSV length : 652
Current CSV length : 653
Current CSV length : 654
Current CSV length : 655
Current CSV length : 656
Current CSV length : 657
Current CSV length : 658
Current CSV length : 659
Current CSV length : 660
Current CSV length : 661
Current CSV length : 662
Current CSV length : 663
Current CSV length : 664
Current CSV length : 665
Current CSV length : 666
Current CSV length : 667
Current CSV length : 668
Current CSV length : 669
Current CSV length : 670
Current CSV length : 671
Current CSV length : 672
Current CSV length : 673
Current CSV length : 674
Current CSV length : 675
Current CSV length : 676
Current CSV length : 677
Current CSV length : 678
Current CSV length : 679
Current CSV length : 680


Current CSV length : 960
Current CSV length : 961
Current CSV length : 962
Current CSV length : 963
Current CSV length : 964
Current CSV length : 965
Current CSV length : 966
Current CSV length : 967
Current CSV length : 968
Current CSV length : 969
Current CSV length : 970
Current CSV length : 971
Current CSV length : 972
Current CSV length : 973
Current CSV length : 974
Current CSV length : 975
Current CSV length : 976
Current CSV length : 977
Current CSV length : 978
Current CSV length : 979
Current CSV length : 980
Current CSV length : 981
Current CSV length : 982
Current CSV length : 983
Current CSV length : 984
Current CSV length : 985
Current CSV length : 986
Current CSV length : 987
Current CSV length : 988
Current CSV length : 989
Current CSV length : 990
Current CSV length : 991
Current CSV length : 992
Current CSV length : 993
Current CSV length : 994
Current CSV length : 995
Current CSV length : 996
Current CSV length : 997
Current CSV length : 998
Current CSV length : 999


Current CSV length : 1270
Current CSV length : 1271
Current CSV length : 1272
Current CSV length : 1273
Current CSV length : 1274
Current CSV length : 1275
Current CSV length : 1276
Current CSV length : 1277
Current CSV length : 1278
Current CSV length : 1279
Current CSV length : 1280
Current CSV length : 1281
Current CSV length : 1282
Current CSV length : 1283
Current CSV length : 1284
Current CSV length : 1285
Current CSV length : 1286
Current CSV length : 1287
Current CSV length : 1288
Current CSV length : 1289
Current CSV length : 1290
Current CSV length : 1291
Current CSV length : 1292
Current CSV length : 1293
Current CSV length : 1294
Current CSV length : 1295
Current CSV length : 1296
Current CSV length : 1297
Current CSV length : 1298
Current CSV length : 1299
Current CSV length : 1300
Current CSV length : 1301
Current CSV length : 1302
Current CSV length : 1303
Current CSV length : 1304
Current CSV length : 1305
Current CSV length : 1306
Current CSV length : 1307
Current CSV 

Current CSV length : 1576
Current CSV length : 1577
Current CSV length : 1578
Current CSV length : 1579
Current CSV length : 1580
Current CSV length : 1581
Current CSV length : 1582
Current CSV length : 1583
Current CSV length : 1584
Current CSV length : 1585
Current CSV length : 1586
Current CSV length : 1587
Current CSV length : 1588
Current CSV length : 1589
Current CSV length : 1590
Current CSV length : 1591
Current CSV length : 1592
Current CSV length : 1593
Current CSV length : 1594
Current CSV length : 1595
Current CSV length : 1596
Current CSV length : 1597
Current CSV length : 1598
Current CSV length : 1599
Current CSV length : 1600
Current CSV length : 1601
Current CSV length : 1602
Current CSV length : 1603
Current CSV length : 1604
Current CSV length : 1605
Current CSV length : 1606
Current CSV length : 1607
Current CSV length : 1608
Current CSV length : 1609
Current CSV length : 1610
Current CSV length : 1611
Current CSV length : 1612


WebDriverException: Message: unknown error: net::ERR_TUNNEL_CONNECTION_FAILED
  (Session info: chrome=129.0.6668.71)
Stacktrace:
	GetHandleVerifier [0x011A7143+25587]
	(No symbol) [0x0113A2E4]
	(No symbol) [0x01032113]
	(No symbol) [0x0102A74B]
	(No symbol) [0x0101E518]
	(No symbol) [0x0101FE5B]
	(No symbol) [0x0101E767]
	(No symbol) [0x0101E10C]
	(No symbol) [0x0101E038]
	(No symbol) [0x0101C488]
	(No symbol) [0x0101CA1B]
	(No symbol) [0x01034B84]
	(No symbol) [0x010B5E37]
	(No symbol) [0x0109AB9C]
	(No symbol) [0x010B5370]
	(No symbol) [0x0109A936]
	(No symbol) [0x0106BA73]
	(No symbol) [0x0106C4CD]
	GetHandleVerifier [0x01484C63+3030803]
	GetHandleVerifier [0x014D6B99+3366473]
	GetHandleVerifier [0x012395F2+624802]
	GetHandleVerifier [0x01240E6C+655644]
	(No symbol) [0x01142C9D]
	(No symbol) [0x0113FD68]
	(No symbol) [0x0113FF05]
	(No symbol) [0x01132336]
	BaseThreadInitThunk [0x768EFCC9+25]
	RtlGetAppContainerNamedObjectPath [0x77C080CE+286]
	RtlGetAppContainerNamedObjectPath [0x77

In [ ]:
        shipping_info_panel = soup_product.find('div', id='accordion__panel-0')
        if shipping_info_panel:
            shipping_info = shipping_info_panel.find_all('p', class_='shippingInfo')
            for info in shipping_info:
                text = info.text.strip()
                # Split by the first colon to separate key and value
                if ':' in text:
                    key, value = text.split(':', 1)
                    product_info[key.strip()] = value.strip()  # Flatten the structure
                else:
                    product_info[text] = None  # General info without a specific label


In [ ]:
with open(csv_file_path, 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['Image URL', 'Brand', 'Product Title', 'Price', 'Location', 'Item Number', 'Year', 'Condition']
    
    # Start with the shipping info fields, if they exist
    sample_product_info = product_data(start_page_url)  # Fetch one product to get potential keys
    shipping_keys = [key for key in sample_product_info if key not in fieldnames]
    fieldnames += shipping_keys  # Add dynamic keys to the fieldnames
    fieldnames.append('Comes With')  # Always add 'Comes With'

    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    all_product_info = []
    current_page = 1

    while True:
        page_url = f"{start_page_url}?page={current_page}"
        driver.get(page_url)
        time.sleep(random.uniform(2, 4))  # Random delay 

        product_urls = extract_product_urls()
        print(f"Found {len(product_urls)} product URLs on page {current_page}.")
        
        for url in product_urls:
            product_info = product_data(url)
            if product_info:
                all_product_info.append(product_info)
                # Write the row, ensuring missing keys are filled with None
                writer.writerow({key: product_info.get(key, None) for key in fieldnames})
                    
            length = len(all_product_info)
            print(f"Current CSV length : {length}")                
            if not product_urls:
                print("No more products found.")
                break

            current_page += 1

        print(f"Data saved to {csv_file_path}")

